Handwritten Digit Recognition is a classic problem in Machine Learning and Computer Vision. It involves recognizing digits from 0 through 9 from images or scanned documents. This project does this task through Neural Networks.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical

The dataset used contains 42,000 rows where each row represents an image.
First column (label) indicated the digit (0-9) and the remaining columns represent the pixel values of the image.
These pixels are separated into x (pixel values) and y (labels).
x has 42,000 samples with 784 features (28x28 pixels), and y has 42,000 labels

In [4]:
train_data = pd.read_csv("data/Train.csv")
print("Shape of train_data:", train_data.shape)

X = train_data.iloc[:, 1:] # Get pixel values
y = train_data.iloc[:, 0] # Get labels

print("Shape of X after separating features:", X.shape)

Shape of train_data: (42000, 785)
Shape of X after separating features: (42000, 784)


We can see that the shape of the data has 